<a href="https://colab.research.google.com/github/yujungha/AI-Study/blob/main/10_2%EC%A3%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [138]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [139]:
!kaggle competitions download -c titanic

gender_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)
train.csv: Skipping, found more recently modified local copy (use --force to force download)


In [140]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import torch.nn as nn

In [141]:
df = pd.read_csv('train.csv')
df2 = df[['Survived', 'Pclass', 'Sex', 'Age', 'Fare']].reset_index()

df2.loc[:,'Sex'] = df2.loc[:,'Sex'].str.replace('female', '0')
df2.loc[:,'Sex'] = df2.loc[:,'Sex'].str.replace('male', '1')
df2['Sex'] = df2['Sex'].astype(int)
df2['Pclass'] = round(df2['Pclass'] / 3, 1)
df2['Age'] = round(df2['Age'] / 80 , 1)
df2 = df2.dropna(subset=['Age'])
df2['Fare'] = round(df2['Fare'] / df2['Fare'].max() , 3)
df3 = df2['Survived'].values
df3 = df3.reshape(-1, 1)
del df2['index']
del df2['Survived']

x_train = torch.FloatTensor(df2.values)
y_train = torch.FloatTensor(df3)

model = nn.Sequential(
    nn.Linear(4, 50),
    nn.ReLU(),
    nn.Linear(50, 50),
    nn.ReLU(),
    nn.Linear(50, 50),
    nn.ReLU(),
    nn.Linear(50, 50),
    nn.ReLU(),
    nn.Linear(50, 50),
    nn.ReLU(),
    nn.Linear(50, 1)
)

learning_rate = 0.01
epochs = 10000

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
cost = nn.MSELoss()

for i in range(epochs + 1):
  y = model(x_train)
  loss = cost(y, y_train)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if i % 1000 == 0:
    output = torch.abs(torch.round(model(x_train)))
    print(i, loss.item(), '%.2f%%'%((y_train == output).float().mean().numpy() * 100))

0 0.419853538274765 59.38%
1000 0.09310165047645569 87.25%
2000 0.09279683977365494 87.54%
3000 0.07627182453870773 88.94%
4000 0.06865232437849045 91.04%
5000 0.06773509085178375 90.76%
6000 0.07147190719842911 90.06%
7000 0.0633544996380806 91.32%
8000 0.062044043093919754 91.74%
9000 0.0627489984035492 91.18%
10000 0.07121502608060837 91.18%


In [74]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [65]:
test_df2[test_df2['Name'].str.contains('Ms')]

,index,Name,Age
88,88,"O'Donoghue, Ms. Bridget",NaN


In [142]:
test_df = pd.read_csv('test.csv')

# test_df2 = test_df[['Name', 'Age']].reset_index()
Mr_mean = test_df[test_df['Name'].str.contains('Mr\.')]['Age'].mean()
test_df[test_df['Name'].str.contains('Mr\.')] = test_df[test_df['Name'].str.contains('Mr\.')].fillna(Mr_mean) #  Mr 나이의 평균으로 결측값 채우기
Mrs_mean = test_df[test_df['Name'].str.contains('Mrs\.')]['Age'].mean()
test_df[test_df['Name'].str.contains('Mrs\.')] = test_df[test_df['Name'].str.contains('Mrs\.')].fillna(Mrs_mean) #  Mrs 나이의 평균으로 결측값 채우기
Master_mean = test_df[test_df['Name'].str.contains('Master')]['Age'].mean()
test_df[test_df2['Name'].str.contains('Master')] = test_df[test_df['Name'].str.contains('Master')].fillna(Master_mean) #  Master 나이의 평균으로 결측값 채우기
Miss_mean = test_df[test_df['Name'].str.contains('Miss')]['Age'].mean()
test_df[test_df['Name'].str.contains('Miss')] = test_df[test_df['Name'].str.contains('Miss')].fillna(Miss_mean) #  Miss 나이의 평균으로 결측값 채우기
# Ms_mean = test_df2[test_df2['Name'].str.contains('Ms')]['Age'].mean()
test_df[test_df['Name'].str.contains('Ms')] = test_df[test_df['Name'].str.contains('Ms')].fillna(Miss_mean) #  Ms 나이의 평균으로 결측값 채우기

test_df = test_df[['Pclass', 'Sex', 'Age', 'Fare']].reset_index()

test_df.loc[:,'Sex'] = test_df.loc[:,'Sex'].str.replace('female', '0').replace('male', '1')
test_df['Sex'] = test_df['Sex'].astype(np.int32)
test_df['Pclass'] = round(test_df['Pclass'] / test_df['Pclass'].max(), 1)
test_df['Age'] = round(test_df['Age'] / test_df['Age'].max() , 1)
test_df['Fare'].fillna(test_df['Fare'].mean(), inplace=True)
test_df['Fare'] = round(test_df['Fare'] / test_df['Fare'].max() , 3)
del test_df['index']

x_test = torch.FloatTensor(test_df[['Pclass', 'Sex', 'Age', 'Fare']].values)

In [143]:
submit_df = pd.read_csv('gender_submission.csv')
submit_df['Survived'] = model(x_test).round().int().numpy()
submit_df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [144]:
submit_df.to_csv('submission.csv', index=False)

In [145]:
!kaggle competitions submit -c titanic -f submission.csv -m "Message"

100% 2.77k/2.77k [00:00<00:00, 13.2kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster